In [3]:
import pandas as pd
import numpy as np
from distance_benchmark import compute_pfipf

### Init : read + pfipf

In [4]:
path_o = "../data/preliminary_phase/output_org_data_TRP/"
path_r = "../data/testing_data/output_ref_osaka/"

In [5]:
o_d1 = np.load(f"{path_o}/cell_traj_d1.npy")
o_d2 = np.load(f"{path_o}/cell_traj_d2.npy")
r_d1 = np.load(f"{path_r}/cell_traj_d1.npy")
r_d2 = np.load(f"{path_r}/cell_traj_d2.npy")

o_d1d2 = np.concatenate([o_d1, o_d2], axis=1)
r_d1d2 = np.concatenate([r_d1, r_d2], axis=1)
assert(o_d1d2.shape == r_d1d2.shape)

In [6]:
pfipf_o = compute_pfipf(o_d1d2, max(o_d1d2.max(), r_d1d2.max()), o_d1d2.shape[0])
pfipf_r = compute_pfipf(r_d1d2, max(o_d1d2.max(), r_d1d2.max()), o_d1d2.shape[0])

### Anonymisation

In [39]:
assert np.count_nonzero(pfipf_o >= 0.3)/2000 >= 0.7

In [50]:
o_d1d2_ano = o_d1d2.copy()
for i in range(o_d1d2_ano.shape[0]):
    for j in range(o_d1d2_ano.shape[1]):
        index = o_d1d2_ano[i][j]
        if pfipf_o[i][index] >= 0.3 or (j < 2 or (o_d1d2_ano.shape[1]/2 <= j < (o_d1d2_ano.shape[1]/2) + 2)):
            o_d1d2_ano[i][j] = np.random.randint(1, 1025)

In [51]:
df_ano = pd.DataFrame(o_d1d2_ano)
df_ano = df_ano.reset_index()
df_ano["index"] = df_ano.index + 1
df_ano = df_ano.melt("index").sort_values(["index", "variable"]).reset_index(drop=True).drop("variable", axis=1)
df_ano = df_ano.rename(columns={"index": "user_id", "value": "reg_id"})

In [42]:
df_ori = pd.read_csv(f"{path_o}/dataframe.csv").drop(["y_id", "x_id", "y(center)", "x(center)", "hospital"], axis=1)
df_ori.head()

,user_id,time_id,reg_id
0,1,41,1016
1,1,42,952
2,1,43,951
3,1,44,951
4,1,45,763


In [43]:
df_ori["reg_id"] = df_ano["reg_id"].astype(str)

In [44]:
df_ori["reg_id"].replace("-1", "*", inplace=True)

In [45]:
df_ori["reg_id"].to_csv("anon/anotraces_team011_data02_TRP.csv", index=False, header="reg_id")